# Section 8 — Data Engineering
Concepts and practical snippets for databases, ETL/ELT, and handling large datasets.

**Topics:** SQL basics, `sqlite3`, SQLAlchemy, (conceptual) BigQuery, ETL/ELT patterns, chunked I/O, and simple parallelization.

## 8.1 Databases & SQL — Concepts
- **Relational Databases (RDBMS):** tables, rows, columns, keys (PK/FK).
- **SQL:** `SELECT`, `WHERE`, `JOIN`, `GROUP BY`, `ORDER BY`, subqueries, window functions.
- **When to use what:**
  - `sqlite3` for local prototyping & tests.
  - **SQLAlchemy** for ORM/connection management.
  - **BigQuery** (or cloud warehouses) for analytics at scale.

Below: runnable `sqlite3` examples and SQLAlchemy basics (with SQLite).

### SQL Cheat Sheet (Quick)

In [ ]:
# Illustrative (not executed as SQL here):
sql_select = '''
SELECT city, SUM(sales) AS total_sales
FROM transactions
WHERE date >= '2025-01-01'
GROUP BY city
ORDER BY total_sales DESC;
'''
print(sql_select)

## 8.2 `sqlite3` — Lightweight Local Database
Create a database in memory, create a table, insert data, and run queries.

In [ ]:
import sqlite3
import pandas as pd

# In-memory database
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create table
cur.execute('''CREATE TABLE transactions (
    id INTEGER PRIMARY KEY,
    city TEXT,
    sales REAL,
    date TEXT
)''')
conn.commit()

# Insert sample rows
rows = [
    (1, 'Montreal', 120.0, '2025-01-01'),
    (2, 'Toronto', 200.0, '2025-01-01'),
    (3, 'Montreal', 170.0, '2025-02-01'),
    (4, 'Vancouver', 150.0, '2025-01-15'),
]
cur.executemany('INSERT INTO transactions VALUES (?, ?, ?, ?)', rows)
conn.commit()

# Query with SQL
query = "SELECT city, SUM(sales) AS total_sales FROM transactions GROUP BY city ORDER BY total_sales DESC;"
df = pd.read_sql_query(query, conn)
df

## 8.3 SQLAlchemy — Engine & ORM Basics (with SQLite)
Use SQLAlchemy to create an engine, execute SQL, or define models (ORM).

In [ ]:
from sqlalchemy import create_engine, text
engine = create_engine('sqlite:///:memory:', echo=False)
with engine.begin() as conn:
    conn.execute(text('CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, city TEXT)'))
    conn.execute(text("INSERT INTO users (id, name, city) VALUES (1, 'Alice', 'Montreal'), (2, 'Bob', 'Toronto')"))
    result = conn.execute(text('SELECT * FROM users'))
    print(result.all())

> **Tip:** With SQLAlchemy ORM you can define Python classes mapped to tables and let the ORM handle CRUD operations. For analytics, many teams use Core/SQL expressions or raw SQL for clarity and performance.

## 8.4 BigQuery (Conceptual)
- **BigQuery** is a serverless, highly scalable cloud data warehouse by Google.
- You can query massive datasets with standard SQL and pay per data scanned.
- Python access typically uses the **`google-cloud-bigquery`** client library and service account credentials.
- Typical flow:
  1) Create a GCP project & dataset; 2) authenticate (env var `GOOGLE_APPLICATION_CREDENTIALS`);
  3) Use the client to run queries / load data from GCS; 4) write results to tables or export to storage.

_Not executed here due to credentials, but snippet shown below for reference._

In [ ]:
# Reference snippet (not executed here):
bigquery_example = r'''
from google.cloud import bigquery
client = bigquery.Client()
query_job = client.query("""
    SELECT city, SUM(sales) AS total_sales
    FROM `project.dataset.transactions`
    WHERE date >= '2025-01-01'
    GROUP BY city
    ORDER BY total_sales DESC
""")
for row in query_job:
    print(row)
'''
print(bigquery_example)

## 8.5 ETL vs ELT — Patterns & Best Practices
- **ETL** (Extract → Transform → Load): transform data before loading into the warehouse.
- **ELT** (Extract → Load → Transform): load raw data first, then transform in the warehouse (common in modern stacks).

**Best Practices:**
- Use idempotent steps (safe re-runs).
- Keep raw data immutable.
- Parameterize configs (env vars / YAML).
- Log & monitor jobs.
- Version your transformations (SQL or code).

### Simple ETL Skeleton (in Python)

In [ ]:
from pathlib import Path
import pandas as pd

RAW = Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
PROC = Path('data/processed'); PROC.mkdir(parents=True, exist_ok=True)

def extract() -> pd.DataFrame:
    # toy extract: create a DataFrame
    return pd.DataFrame({
        'city': ['Montreal','Toronto','Montreal'],
        'sales': [120, 200, 170],
        'date': ['2025-01-01','2025-01-01','2025-02-01']
    })

def transform(df: pd.DataFrame) -> pd.DataFrame:
    df['date'] = pd.to_datetime(df['date'])
    return df

def load(df: pd.DataFrame, path: Path):
    df.to_csv(path, index=False)

df_raw = extract()
df_t = transform(df_raw)
load(df_t, PROC / 'transactions.csv')
df_t

## 8.6 Chunked I/O — Handling Larger Files in Pandas
Read and process large CSVs in chunks to reduce memory usage.

In [ ]:
import pandas as pd
from pathlib import Path

# Create a sample CSV with many rows (synthetic, small here)
path = Path('data/processed/large_sample.csv')
pd.DataFrame({'x': range(1000), 'y': range(1000, 2000)}).to_csv(path, index=False)

total = 0
for chunk in pd.read_csv(path, chunksize=200):
    total += chunk['y'].sum()
total

## 8.7 Simple Parallelization Example (CPU-bound)
Use `multiprocessing` to parallelize CPU-bound tasks (e.g., transformations).

In [ ]:
from multiprocessing import Pool, cpu_count
import math

def heavy_fn(n: int) -> float:
    # Example: compute something moderately heavy
    return sum(math.sqrt(i) for i in range(n))

with Pool(processes=min(4, cpu_count())) as pool:
    results = pool.map(heavy_fn, [10_000, 20_000, 30_000, 40_000])
results[:2]

> **Notes**
- For large-scale data, consider distributed tools (e.g., **Dask**, **Spark**).
- Profile I/O vs CPU bottlenecks; optimize accordingly.
- Use columnar formats (Parquet) for analytics workloads.